In [0]:
import os
os.environ['SPARK_VERSION'] = '3.3'

import pydeequ
from pydeequ.checks import *
from pydeequ.verification import *
profiles_df = spark.read.table("default.user_behavior_profiles")

In [0]:
# Conjunto de testes
check = Check(spark, CheckLevel.Error, "Verificações de Qualidade da Tabela de Perfis")

verification_suite_result = VerificationSuite(spark) \
    .onData(profiles_df) \
    .addCheck(
        check.isComplete("user_id") \
             .isUnique("user_id") \
             .isNonNegative("avg_amount") \
             .isNonNegative("stddev_amount") \
             .isNonNegative("transaction_count") \
             .hasSize(lambda size: size > 0) # Garante que a tabela não está vazia
    ) \
    .run()

result_df = VerificationResult.checkResultsAsDataFrame(spark, verification_suite_result)

display(result_df)

In [0]:
if verification_suite_result.status == "Success":
    print("Todos os testes de qualidade de dados passaram com sucesso!")
else:
    print("Falha nos testes de qualidade de dados. Verifique o resultado acima.")
    raise Exception("Pipeline de qualidade de dados falhou. Verifique os logs.")